# Train model

In [ ]:
from scripts.train import run_lgbm_nested_training
TRAIN_1_PATH = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X1_all_rank_features_with_y.tsv"
TRAIN_2_PATH   = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X2_all_rank_features_with_y.tsv"
TEST_PATH = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X3_test.tsv"
FEATURES_PATH = "./Code submission/top_features_shap/selected_union_features.tsv"
OUTPUT_DIR = "./results/lgbm/mix_X1_X2/"

results_df, results_inner_df, test_predictions_df,_ = run_lgbm_nested_training(
        train_1_path=TRAIN_1_PATH,
        train_2_path=TRAIN_2_PATH,
        test_path=TEST_PATH,
        features_path=FEATURES_PATH,
        output_dir=OUTPUT_DIR,
        seed=42,
        n_inner_folds=5,
        meta_cols=["gene_name", "chr", "gene_start", "gene_end",
                   "TSS_start", "TSS_end", "strand", "gex", "gex_rank"],
    )

# ============================================================
# SUMMARY
# ============================================================

# # Optional: save both
results_df.to_csv(f"{OUTPUT_DIR}/results_summary.tsv", sep="\t", index=False)
results_inner_df.to_csv(f"{OUTPUT_DIR}/results_inner_folds.tsv", sep="\t", index=False)
test_predictions_df.to_csv(f"{OUTPUT_DIR}/gex_predicted.tsv", sep="\t", index=False)
print("\n💾 Saved detailed results → results_inner_folds.tsv")
print("\n💾 Saved test predictions → gex_predicted.tsv")


🚀 Starting nested LGBM training pipeline
🔑 Loaded 63 selected features from ../preprocessed_data/reference/1. merged data/with_y_100_one_side/shap_results/top70_features_pruned/selected_union_features.tsv
🔀 Merged TRAIN 1 and TRAIN 2 for training.
Chromosomes: ['chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22']
✅ Config saved to ../results/final/lgbm/mix_X1_X2//config.json

🚀 Leave-one-chromosome: chr2
🧩 Chromosome folds:
Fold 1: ['chr3', 'chr8', 'chr13', 'chr18']
 → Validation fold 1 (chr3,chr8,chr13,chr18): ρ_reg=0.8188, ρ_reg_masked=0.7939
 → Predict chr-kfold chr2 in 1: ρ_reg=0.7933, ρ_reg_masked=0.7727
Fold 2: ['chr4', 'chr9', 'chr14', 'chr19']
 → Validation fold 2 (chr4,chr9,chr14,chr19): ρ_reg=0.8135, ρ_reg_masked=0.7883
 → Predict chr-kfold chr2 in 2: ρ_reg=0.7915, ρ_reg_masked=0.7775
Fold 3: ['chr5', 'chr10', 'chr15', 'chr20']
 → Validation fold 3 (

In [35]:
from scripts.train import run_leaveonechr_with_innerkfold

TRAIN_1_PATH = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X1_all_rank_features_with_y.tsv"
TRAIN_2_PATH   = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X2_all_rank_features_with_y.tsv"
TEST_PATH = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X3_test.tsv"
FEATURES_PATH = "./top_features_shap/selected_union_features.tsv"
OUTPUT_DIR = "./results/lgbm/Only_X1/"

META_COLS = [
        "gene_name", "chr", "gene_start", "gene_end",
        "TSS_start", "TSS_end", "strand", "gex", "gex_rank"
    ]

df_outer, df_inner, _, unlabeled_pred = run_leaveonechr_with_innerkfold(
            train_path=TRAIN_1_PATH,
            test_path=TRAIN_2_PATH,
            unlabeled_test_path=TEST_PATH,
            features_path=FEATURES_PATH,
            output_dir=OUTPUT_DIR,
            meta_cols=META_COLS,
            seed=42,
            n_inner_folds=5,
            THRESHOLD=0.4,
        )

# ============================================================
# SUMMARY
# ============================================================


print(df_inner.head())

# # Optional: save both
df_outer.to_csv(f"{OUTPUT_DIR}/results_summary.tsv", sep="\t", index=False)
df_inner.to_csv(f"{OUTPUT_DIR}/results_inner_folds.tsv", sep="\t", index=False)
unlabeled_pred.to_csv(f"{OUTPUT_DIR}/gex_predicted.tsv", sep="\t", index=False)
print("\n💾 Saved detailed results → results_inner_folds.tsv")
print("\n💾 Saved test predictions → gex_predicted.tsv")


🔒 Global seed set to 42
🧪 Loaded unlabeled test set: ../preprocessed_data/reference/1. merged data/with_y_100_one_side/X3_test.tsv (shape=(1984, 2920))
✅ Loaded 63 selected features from ./top_features_shap/selected_union_features.tsv
💾 Config saved → ./results/lgbm/Only_X1/config.json
Chromosomes: ['chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22']

🚀 Outer Fold = Leave out chr2
  ⚙️ Inner Fold 1 | val_chrs=['chr3', 'chr8', 'chr13', 'chr18']
     chr-kfold Val:['chr3', 'chr8', 'chr13', 'chr18'] ρ_nomask=0.8040, ρ_masked=0.8376
     chr-kfold Loco chr2: ρ_nomask=0.7941, ρ_masked=0.8303
     chr-kfold Test: ρ_nomask=0.8009, ρ_masked=0.8156
     chr-kfold Test ONLY chr2: ρ_nomask=0.7906, ρ_masked=0.8010
  ⚙️ Inner Fold 2 | val_chrs=['chr4', 'chr9', 'chr14', 'chr19']
     chr-kfold Val:['chr4', 'chr9', 'chr14', 'chr19'] ρ_nomask=0.8052, ρ_masked=0.8317
     chr

In [17]:
from scripts.train import run_leaveonechr_with_innerkfold
TRAIN_1_PATH = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X1_all_rank_features_with_y.tsv"
TRAIN_2_PATH   = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X2_all_rank_features_with_y.tsv"
TEST_PATH = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X3_test.tsv"
FEATURES_PATH = "./top_features_shap/selected_union_features.tsv"
OUTPUT_DIR = "./results/lgbm/Only_X2_try/"

META_COLS = [
        "gene_name", "chr", "gene_start", "gene_end",
        "TSS_start", "TSS_end", "strand", "gex", "gex_rank"
    ]

df_outer, df_inner, _, unlabeled_pred = run_leaveonechr_with_innerkfold(
            train_path=TRAIN_2_PATH,
            test_path=TRAIN_1_PATH,
            unlabeled_test_path=TEST_PATH,
            features_path=FEATURES_PATH,
            output_dir=OUTPUT_DIR,
            meta_cols=META_COLS,
            seed=42,
            n_inner_folds=5,
            THRESHOLD=0.4,
        )

# ============================================================
# SUMMARY
# ============================================================


print(df_inner.head())

# # Optional: save both
df_outer.to_csv(f"{OUTPUT_DIR}/results_summary.tsv", sep="\t", index=False)
df_inner.to_csv(f"{OUTPUT_DIR}/results_inner_folds.tsv", sep="\t", index=False)
unlabeled_pred.to_csv(f"{OUTPUT_DIR}/gex_predicted.tsv", sep="\t", index=False)
print("\n💾 Saved detailed results → results_inner_folds.tsv")
print("\n💾 Saved test predictions → gex_predicted.tsv")


🔒 Global seed set to 42
🧪 Loaded unlabeled test set: ../preprocessed_data/reference/1. merged data/with_y_100_one_side/X3_test.tsv (shape=(1984, 2920))
✅ Loaded 63 selected features from ./top_features_shap/selected_union_features.tsv
💾 Config saved → ./results/lgbm/Only_X2_try/config.json
Chromosomes: ['chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22']

🚀 Outer Fold = Leave out chr2
  ⚙️ Inner Fold 1 | val_chrs=['chr3', 'chr8', 'chr13', 'chr18']
     chr-kfold Val:['chr3', 'chr8', 'chr13', 'chr18'] ρ_nomask=0.8400, ρ_masked=0.8526
     chr-kfold Loco chr2: ρ_nomask=0.8223, ρ_masked=0.8374
     chr-kfold Test: ρ_nomask=0.7839, ρ_masked=0.8147
     chr-kfold Test ONLY chr2: ρ_nomask=0.7798, ρ_masked=0.8132
  ⚙️ Inner Fold 2 | val_chrs=['chr4', 'chr9', 'chr14', 'chr19']
     chr-kfold Val:['chr4', 'chr9', 'chr14', 'chr19'] ρ_nomask=0.8401, ρ_masked=0.8520
    

In [ ]:
from scripts.train import loco_chr_kfold_xcell_lgbm
TRAIN_1_PATH = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X1_all_rank_features_with_y.tsv"
TRAIN_2_PATH   = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X2_all_rank_features_with_y.tsv"
TEST_PATH = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X3_test.tsv"
FEATURES_PATH = "./top_features_shap/selected_union_features.tsv"
OUTPUT_DIR = "./results/lgbm/Only_X1_val_on_X2/"

META_COLS = [
        "gene_name", "chr", "gene_start", "gene_end",
        "TSS_start", "TSS_end", "strand", "gex", "gex_rank"
    ]

df_outer, df_inner, unlabeled_pred = loco_chr_kfold_xcell_lgbm(
            train_path=TRAIN_1_PATH,
            test_path=TRAIN_2_PATH,
            unlabeled_test_path=TEST_PATH,
            features_path=FEATURES_PATH,
            output_dir=OUTPUT_DIR,
            meta_cols=META_COLS,
            seed=42,
            n_inner_folds=5,
            THRESHOLD=0.4,
        )

# ============================================================
# SUMMARY
# ============================================================


print(df_inner.head())

# # Optional: save both
df_outer.to_csv(f"{OUTPUT_DIR}/results_summary.tsv", sep="\t", index=False)
df_inner.to_csv(f"{OUTPUT_DIR}/results_inner_folds.tsv", sep="\t", index=False)
unlabeled_pred.to_csv(f"{OUTPUT_DIR}/gex_predicted.tsv", sep="\t", index=False)
print("\n💾 Saved detailed results → results_inner_folds.tsv")
print("\n💾 Saved test predictions → gex_predicted.tsv")


🔒 Global seed set to 42
🧪 Loaded unlabeled test set: ../preprocessed_data/reference/1. merged data/with_y_100_one_side/X3_test.tsv (shape=(1984, 2920))
✅ Loaded 63 selected features from ./top_features_shap/selected_union_features.tsv
💾 Config saved → ./results/lgbm/Only_X1_val_on_X2/config.json
Chromosomes: ['chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22']

🚀 Outer Fold = Leave out chr2
  ⚙️ Inner Fold 1 | val_chrs=['chr3', 'chr8', 'chr13', 'chr18']
     chr-kfold on test Val:['chr3', 'chr8', 'chr13', 'chr18'] ρ_nomask=0.8011, ρ_masked=0.8172
     chr-kfold on train Val:['chr3', 'chr8', 'chr13', 'chr18'] ρ_nomask=0.8037, ρ_masked=0.8395
     chr-kfold Loco in Test chr2: ρ_nomask=0.7914, ρ_masked=0.8069
     chr-kfold Loco in Train chr2: ρ_nomask=0.7928, ρ_masked=0.8282
  ⚙️ Inner Fold 2 | val_chrs=['chr4', 'chr9', 'chr14', 'chr19']
     chr-kfold on test 

In [31]:
from scripts.train import loco_chr_kfold_xcell_lgbm
TRAIN_1_PATH = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X1_all_rank_features_with_y.tsv"
TRAIN_2_PATH   = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X2_all_rank_features_with_y.tsv"
TEST_PATH = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X3_test.tsv"
FEATURES_PATH = "./top_features_shap/selected_union_features.tsv"
OUTPUT_DIR = "./results/lgbm/Only_X2_val_on_X1/"

META_COLS = [
        "gene_name", "chr", "gene_start", "gene_end",
        "TSS_start", "TSS_end", "strand", "gex", "gex_rank"
    ]

df_outer, df_inner, unlabeled_pred = loco_chr_kfold_xcell_lgbm(
            train_path=TRAIN_2_PATH,
            test_path=TRAIN_1_PATH,
            unlabeled_test_path=TEST_PATH,
            features_path=FEATURES_PATH,
            output_dir=OUTPUT_DIR,
            meta_cols=META_COLS,
            seed=42,
            n_inner_folds=5,
            THRESHOLD=0.4,
        )

# ============================================================
# SUMMARY
# ============================================================


print(df_inner.head())

# # Optional: save both
df_outer.to_csv(f"{OUTPUT_DIR}/results_summary.tsv", sep="\t", index=False)
df_inner.to_csv(f"{OUTPUT_DIR}/results_inner_folds.tsv", sep="\t", index=False)
unlabeled_pred.to_csv(f"{OUTPUT_DIR}/gex_predicted.tsv", sep="\t", index=False)
print("\n💾 Saved detailed results → results_inner_folds.tsv")
print("\n💾 Saved test predictions → gex_predicted.tsv")


🔒 Global seed set to 42
🧪 Loaded unlabeled test set: ../preprocessed_data/reference/1. merged data/with_y_100_one_side/X3_test.tsv (shape=(1984, 2920))
✅ Loaded 63 selected features from ./top_features_shap/selected_union_features.tsv
💾 Config saved → ./results/lgbm/Only_X2_val_on_X1/config.json
Chromosomes: ['chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22']

🚀 Outer Fold = Leave out chr2
  ⚙️ Inner Fold 1 | val_chrs=['chr3', 'chr8', 'chr13', 'chr18']
     chr-kfold on test Val:['chr3', 'chr8', 'chr13', 'chr18'] ρ_nomask=0.7846, ρ_masked=0.8165
     chr-kfold on train Val:['chr3', 'chr8', 'chr13', 'chr18'] ρ_nomask=0.8396, ρ_masked=0.8526
     chr-kfold Loco in Test chr2: ρ_nomask=0.7797, ρ_masked=0.8168
     chr-kfold Loco in Train chr2: ρ_nomask=0.8215, ρ_masked=0.8359
  ⚙️ Inner Fold 2 | val_chrs=['chr4', 'chr9', 'chr14', 'chr19']
     chr-kfold on test 

# Merge the prediction into the final file

In [4]:
from scripts.train import aggregate_gex_predicted
df = aggregate_gex_predicted(
    dirs_pred=[
        "./result/lgbm/Only_X1",
        "./result/lgbm/Only_X2",
        "./result/lgbm/Only_X1_val_on_X2",
        "./result/lgbm/Only_X2_val_on_X1",
        "./result/lgbm/mix_X1_X2",
    ],
    dirs_prob=[
        "./result/lgbm/Only_X1",
        "./result/lgbm/Only_X2",
        "./result/lgbm/Only_X1_val_on_X2",
        "./result/lgbm/Only_X2_val_on_X1",
                "./result/lgbm/mix_X1_X2",

    ],
    threshold=0.4,
    output_dir="./result/lgbm/ensemble/",
    zip_name="Wang_Ding_Yang_Project1.zip",
    reference_tsv_path="../ML4G_Project_1_Data/CAGE-train/CAGE-train/X3_test_info.tsv",
)
print(df.head())

[INFO] Reading: ./result/lgbm/Only_X1/gex_predicted.tsv
[INFO] Reading: ./result/lgbm/Only_X2/gex_predicted.tsv
[INFO] Reading: ./result/lgbm/Only_X1_val_on_X2/gex_predicted.tsv
[INFO] Reading: ./result/lgbm/Only_X2_val_on_X1/gex_predicted.tsv
[INFO] Reading: ./result/lgbm/mix_X1_X2/gex_predicted.tsv
[INFO] Reading: ./result/lgbm/Only_X1/gex_predicted.tsv
[INFO] Reading: ./result/lgbm/Only_X2/gex_predicted.tsv
[INFO] Reading: ./result/lgbm/Only_X1_val_on_X2/gex_predicted.tsv
[INFO] Reading: ./result/lgbm/Only_X2_val_on_X1/gex_predicted.tsv
[INFO] Reading: ./result/lgbm/mix_X1_X2/gex_predicted.tsv
[INFO] Saved TSV to: result/lgbm/ensemble/gex_predicted.tsv
[INFO] Saved ZIP to: result/lgbm/ensemble/Wang_Ding_Yang_Project1.zip
[INFO] Aggregated preds from 5 dir(s); probs from 5 dir(s).
[INFO] Output rows (genes): 1984
[INFO] Mask ON rate (@0.400): 44.41%
    gene_name  gex_predicted
0       CAPN9       0.000000
1        ILF2       0.460985
2  ST6GALNAC5       0.000000
3  MROH7-TTC4       